In [1]:
import teradata
import pandas as pd
import matplotlib.pyplot as plt
host,user,pwd ='10.132.4.11','ID', "PW"
udaExec =teradata.UdaExec (appName="HelloWorld", version="1.0",logConsole=False)
with udaExec.connect(method="ODBC", system=host,username=user,password=pwd) as session:
    query1 = '''SELECT * from DD_RO_ADHOC.MERCHANT_RECOM_FINAL5'''#  sample 1000000
    MERCHANT = pd.read_sql(query1,session) 
MERCHANT.shape 

(1328565, 2)

In [6]:
MERCHANTS=MERCHANT [['PARTY_NUMR','MERCHANT_NAME']]
MERCHANTS.head()

,PARTY_NUMR,MERCHANT_NAME
0,11748709,LOH GUAN LYE SPECIALISTS PENANG
1,1511908,AGODA INTERNATION
2,367199,GAMA S
3,12875900,COSWAY
4,1382291,HONG KONG SHENG KEE DESS SEPANG


In [7]:
MERCHANTS['MERCHANT_NAME'].value_counts().head(5)

PARKSON                                   22791
AGODA HOTEL RESERVATIO                    18593
GHL                                       15816
PADINI CONCEPT STORE                      15589
WECHAT PA                                 12097
Name: MERCHANT_NAME, dtype: int64

In [8]:
Mer=[ i.rstrip() for i in MERCHANTS['MERCHANT_NAME']]
#Mer=[]
#for i in MERCHANTS['MERCHANT_NAME']:
#    Mer.append(i.rstrip())
Mer
MERCHANTS['MERCHANT_NAME']=Mer
MERCHANTS
df = MERCHANTS.groupby('PARTY_NUMR').agg({'MERCHANT_NAME': ', '.join}).reset_index()
df[['MERCHANT_NAME']].head()

C:\Users\tvimal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,MERCHANT_NAME
0,"COFFEE BEAN, PRUBSN GT, 0, VILLAGE GROCER, MPH..."
1,"JAYA GROCER SUBANG EMPIR SUBANG JAYA, JAYA GRO..."
2,JAYA GROCER MUTIARA TRPC PETALING JAYA
3,"JAKEL TRADING, PRUBSN GT, STARBUCKS, TAKAFUL M..."
4,"MPH BOOKSTORES, ICHIBAN RAMEN"


In [9]:
lst=list(df.MERCHANT_NAME)
lst
list_of_list=[s.split(',') for s in lst]
len(list_of_list)

434307

In [11]:
from apyori import apriori
association_rules = apriori(list_of_list, min_support=0.001, min_confidence=0.05, min_lift=5, min_length=2)  
association_results = list(association_rules)
association_results

[RelationRecord(items=frozenset({' AGODA HOTEL RESERVATIO', ' MALAY AI'}), support=0.001706166375398048, ordered_statistics=[OrderedStatistic(items_base=frozenset({' AGODA HOTEL RESERVATIO'}), items_add=frozenset({' MALAY AI'}), confidence=0.061708860759493674, lift=6.283842951904671), OrderedStatistic(items_base=frozenset({' MALAY AI'}), items_add=frozenset({' AGODA HOTEL RESERVATIO'}), confidence=0.17373974208675266, lift=6.283842951904671)]),
 RelationRecord(items=frozenset({' BIGPAY', ' BIGPA'}), support=0.0016693260757943114, ordered_statistics=[OrderedStatistic(items_base=frozenset({' BIGPA'}), items_add=frozenset({' BIGPAY'}), confidence=0.21765235664965477, lift=67.47176449638944), OrderedStatistic(items_base=frozenset({' BIGPAY'}), items_add=frozenset({' BIGPA'}), confidence=0.5174875089221984, lift=67.47176449638944)]),
 RelationRecord(items=frozenset({' BOOST APP', ' BIGPA'}), support=0.0011029064693868623, ordered_statistics=[OrderedStatistic(items_base=frozenset({' BIGPA'}

In [13]:
association_results
Result = pd.DataFrame(columns=('Items','Support','Confidence','Lift'))#'Antecedent','Consequent',

Support =[]
Confidence = []
Lift = []
Items = []
#Antecedent = []
#Consequent=[]

for RelationRecord in association_results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
       # Antecedent.append(ordered_stat.items_base)
       #Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)

Result['Items'] =Items# list(map(set, Items))                                   
#df['Antecedent'] = list(map(set, Antecedent))
#df['Consequent'] = list(map(set, Consequent))
Result['Support'] = Support
Result['Confidence'] = Confidence
Result['Lift']= Lift
Result
Result.sort_values(by ='Lift', ascending = False, inplace = True)
print(df.shape)
Result['Items']=[list(x) for x in list(Result['Items'])]


(434307, 2)


In [15]:
Result.head()

,Items,Support,Confidence,Lift
27,"[ VILLAGE GROCER, JAYA GROCER]",0.001216,0.087000,9.651212
26,"[ VILLAGE GROCER, JAYA GROCER]",0.001216,0.134866,9.651212
34,"[ SUSHI ZANMAI, STARBUCKS]",0.001036,0.084112,9.328523
35,"[ SUSHI ZANMAI, STARBUCKS]",0.001036,0.114913,9.328523
13,"[ COTTON ON, SEPHORA]",0.001110,0.128294,7.740889


In [ ]:
df.to_excel('C:\\Users\\tvimal\\Desktop\\Recomended_new.xlsx')

In [11]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

TE = TransactionEncoder()
TE_ary = TE.fit(list_of_list).transform(list_of_list)
dfd = pd.DataFrame(TE_ary, columns=TE.columns_)
dfd.head()

,*MYR-ANYTIME-EZYPAYV2,*THE CHINA PRESS-1,0,11STREET,2C2P*ZARA COM,2C2P-CHILINDO,2C2P-JOMPARKING,2C2P-TRAVEL RECOMMEND,A HONG,AA PHARMACY,...,YTL COMMS,YUBISO,YUNG FA RETAIL,ZALORA MALAYSIA,ZARA,ZENXIN ORGANIC FOOD,ZENZU,Zalora SEA,doTERRA Malaysia,uCloudlink
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
from mlxtend.frequent_patterns import apriori
apriori(dfd, min_support=0.01, use_colnames=True)

,support,itemsets
0,0.027649,( AGODA HOTEL RESERVATIO)
1,0.015735,( BRANDS OUTLET)
2,0.016574,( COTTON ON)
3,0.011572,( FAVE)
4,0.027144,( GHL)
5,0.011197,( IKEA)
6,0.013373,( IKEA DAMANSARA)
7,0.016198,( MOLPAY)
8,0.012307,( MYDIN)
9,0.010893,( NANDO)
